# Create NYC TLC Parquet files

There is New York City Taxi and Limousine Commission (TLC) Trip Record Data stored in S3, [see here](https://registry.opendata.aws/nyc-tlc-trip-records-pds/).

This notebook reads in the CSV data and writes out Parquet files that are easier and more performant to work with.

In [1]:
import coiled
import dask
import dask.dataframe as dd
import pandas as pd

In [2]:
from coiled.v2 import Cluster

cluster = Cluster(
    name="nyc-tlc-cleaning", 
    n_workers=25,
    software='coiled-examples/dask-dataframes',
)

╭───────────────────────────────────────── Coiled Cluster ─────────────────────────────────────────╮
│                     ]8;id=530577;https://cloud.coiled.io/rrpelgrim/clusters/14613/details\https://cloud.coiled.io/rrpelgrim/clusters/14613/details]8;;\                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─────────────────── Overview ───────────────────╮╭──────────────── Configuration ─────────────────╮
│                                                ││                                                │
│ Cluster Name: nyc-tlc-cleaning                 ││ Region: us-east-1                              │
│                                                ││                                                │
│ Cluster Status: ready                          ││ Scheduler Instance Type: t3.medium             │
│                                                ││                                                │
│ Scheduler Status: started                      ││ Worker Instance Type(s): t3.medium (25)        │
│                                                ││                                                │
│ Dashboard Address: ]8;id=118222;http://44.201.64.146:8787\http://44.201.64.146:8787]8;;\   ││ Workers Requested: 25                          │
│                                                ││                                                │
│                                                ││                                                │
╰────────────────────────────────────────────────╯╰────────────────────────────────────────────────╯
╭─────────────────────────────────── (2022/04/26 14:46:23 CEST) ───────────────────────────────────╮
│                                                                                                  │
│                                        All workers ready.                                        │
│                                                                                                  │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

In [3]:
client = dask.distributed.Client(cluster)

/Users/rpelgrim/mambaforge/envs/dask-dataframe/lib/python3.9/site-packages/distributed/client.py:1096: VersionMismatchWarning: Mismatched versions found

+---------+----------------+---------------+---------------+
| Package | client         | scheduler     | workers       |
+---------+----------------+---------------+---------------+
| msgpack | 1.0.3          | 1.0.2         | 1.0.2         |
| python  | 3.9.12.final.0 | 3.9.7.final.0 | 3.9.7.final.0 |
+---------+----------------+---------------+---------------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [17]:
client.restart()

<Client: 'tls://172.18.0.2:8786' processes=0 threads=0, memory=0 B>

## 2009 data create

In [18]:
ddf = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2009-*.csv",
    parse_dates=["Trip_Pickup_DateTime", "Trip_Dropoff_DateTime"],
    dtype={
        "Tolls_Amt": "float64",
        "store_and_forward": "object",
    },
)

In [19]:
ddf.head()

,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,Rate_Code,store_and_forward,End_Lon,End_Lat,Payment_Type,Fare_Amt,surcharge,mta_tax,Tip_Amt,Tolls_Amt,Total_Amt
0,VTS,2009-01-04 02:52:00,2009-01-04 03:02:00,1,2.63,-73.991957,40.721567,NaN,NaN,-73.993803,40.695922,CASH,8.9,0.5,NaN,0.00,0.0,9.40
1,VTS,2009-01-04 03:31:00,2009-01-04 03:38:00,3,4.55,-73.982102,40.736290,NaN,NaN,-73.955850,40.768030,Credit,12.1,0.5,NaN,2.00,0.0,14.60
2,VTS,2009-01-03 15:43:00,2009-01-03 15:57:00,5,10.35,-74.002587,40.739748,NaN,NaN,-73.869983,40.770225,Credit,23.7,0.0,NaN,4.74,0.0,28.44
3,DDS,2009-01-01 20:52:58,2009-01-01 21:14:00,1,5.00,-73.974267,40.790955,NaN,NaN,-73.996558,40.731849,CREDIT,14.9,0.5,NaN,3.05,0.0,18.45
4,DDS,2009-01-24 16:18:23,2009-01-24 16:24:56,1,0.40,-74.001580,40.719382,NaN,NaN,-74.008378,40.720350,CASH,3.7,0.0,NaN,0.00,0.0,3.70


#### Clean Rate_Code NaNs

In [20]:
# # clean rate_code column NaNs
# import math

# def rate_code_to_one(something):
#     if math.isnan(something):
#         return 1
#     else:
#         return something
    
# ddf = ddf.assign(
#     Rate_Code=ddf.Rate_Code.apply(rate_code_to_one, meta=("float64", "int64"))
# )

In [21]:
# vendor_id                     object
# pickup_datetime       datetime64[ns]
# dropoff_datetime      datetime64[ns]
# passenger_count                int64
# trip_distance                float64
# pickup_longitude             float64
# pickup_latitude              float64
# rate_code                      int64
# store_and_fwd_flag            object
# dropoff_longitude            float64
# dropoff_latitude             float64
# payment_type                  object
# fare_amount                  float64
# surcharge                    float64
# mta_tax                      float64
# tip_amount                   float64
# tolls_amount                 float64
# total_amount                 float64
# dtype: object
ddf.dtypes

vendor_name                      object
Trip_Pickup_DateTime     datetime64[ns]
Trip_Dropoff_DateTime    datetime64[ns]
Passenger_Count                   int64
Trip_Distance                   float64
Start_Lon                       float64
Start_Lat                       float64
Rate_Code                       float64
store_and_forward                object
End_Lon                         float64
End_Lat                         float64
Payment_Type                     object
Fare_Amt                        float64
surcharge                       float64
mta_tax                         float64
Tip_Amt                         float64
Tolls_Amt                       float64
Total_Amt                       float64
dtype: object

In [22]:
# rename columns to standardize schema
ddf = ddf.rename(
    columns={
        "vendor_name": "vendor_id",
        "Trip_Pickup_DateTime": "pickup_datetime",
        "Trip_Dropoff_DateTime": "dropoff_datetime",
        "Passenger_Count": "passenger_count",
        "Trip_Distance": "trip_distance",
        "Start_Lon": "pickup_longitude",
        "Start_Lat": "pickup_latitude",
        "Rate_Code": "rate_code",
        "store_and_forward": "store_and_fwd_flag",
        "End_Lon": "dropoff_longitude",
        "End_Lat": "dropoff_latitude",
        "Payment_Type": "payment_type",
        "Fare_Amt": "fare_amount",
        "surcharge": "surcharge",
        "mta_tax": "mta_tax",
        "Tip_Amt": "tip_amount",
        "Tolls_Amt": "tolls_amount",
        "Total_Amt": "total_amount",
    }
)

In [23]:
ddf.dtypes

vendor_id                     object
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count                int64
trip_distance                float64
pickup_longitude             float64
pickup_latitude              float64
rate_code                    float64
store_and_fwd_flag            object
dropoff_longitude            float64
dropoff_latitude             float64
payment_type                  object
fare_amount                  float64
surcharge                    float64
mta_tax                      float64
tip_amount                   float64
tolls_amount                 float64
total_amount                 float64
dtype: object

In [24]:
len(ddf.dtypes)

18

In [25]:
ddf.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,VTS,2009-01-04 02:52:00,2009-01-04 03:02:00,1,2.63,-73.991957,40.721567,NaN,NaN,-73.993803,40.695922,CASH,8.9,0.5,NaN,0.00,0.0,9.40
1,VTS,2009-01-04 03:31:00,2009-01-04 03:38:00,3,4.55,-73.982102,40.736290,NaN,NaN,-73.955850,40.768030,Credit,12.1,0.5,NaN,2.00,0.0,14.60
2,VTS,2009-01-03 15:43:00,2009-01-03 15:57:00,5,10.35,-74.002587,40.739748,NaN,NaN,-73.869983,40.770225,Credit,23.7,0.0,NaN,4.74,0.0,28.44
3,DDS,2009-01-01 20:52:58,2009-01-01 21:14:00,1,5.00,-73.974267,40.790955,NaN,NaN,-73.996558,40.731849,CREDIT,14.9,0.5,NaN,3.05,0.0,18.45
4,DDS,2009-01-24 16:18:23,2009-01-24 16:24:56,1,0.40,-74.001580,40.719382,NaN,NaN,-74.008378,40.720350,CASH,3.7,0.0,NaN,0.00,0.0,3.70


In [26]:
ddf.npartitions

478

In [27]:
ddf.known_divisions

False

In [28]:
ddf.repartition(partition_size="100MB").to_parquet(
    "s3://coiled-datasets/dask-book/nyc-tlc/2009",
    engine="pyarrow",
    compression="snappy",
    write_metadata_file=False,
)

KeyboardInterrupt: 

## 2009 data query

In [12]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/dask-book/nyc-tlc/2009",
    engine="pyarrow",
)

In [13]:
ddf.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,VTS,2009-01-04 02:52:00,2009-01-04 03:02:00,1,2.63,-73.991957,40.721567,1,None,-73.993803,40.695922,CASH,8.9,0.5,NaN,0.00,0.0,9.40
1,VTS,2009-01-04 03:31:00,2009-01-04 03:38:00,3,4.55,-73.982102,40.736290,1,None,-73.955850,40.768030,Credit,12.1,0.5,NaN,2.00,0.0,14.60
2,VTS,2009-01-03 15:43:00,2009-01-03 15:57:00,5,10.35,-74.002587,40.739748,1,None,-73.869983,40.770225,Credit,23.7,0.0,NaN,4.74,0.0,28.44
3,DDS,2009-01-01 20:52:58,2009-01-01 21:14:00,1,5.00,-73.974267,40.790955,1,None,-73.996558,40.731849,CREDIT,14.9,0.5,NaN,3.05,0.0,18.45
4,DDS,2009-01-24 16:18:23,2009-01-24 16:24:56,1,0.40,-74.001580,40.719382,1,None,-74.008378,40.720350,CASH,3.7,0.0,NaN,0.00,0.0,3.70


In [14]:
dtypes_2009 = ddf.dtypes

In [15]:
dtypes_2009

vendor_id                     object
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count                int64
trip_distance                float64
pickup_longitude             float64
pickup_latitude              float64
rate_code                      int64
store_and_fwd_flag            object
dropoff_longitude            float64
dropoff_latitude             float64
payment_type                  object
fare_amount                  float64
surcharge                    float64
mta_tax                      float64
tip_amount                   float64
tolls_amount                 float64
total_amount                 float64
dtype: object

In [38]:
len(ddf)

170896055

In [42]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/nyc-tlc/2009", 
    engine="pyarrow", 
    columns=["Fare_Amt"]
)

In [44]:
%%time
ddf.Fare_Amt.mean().compute()

CPU times: user 124 ms, sys: 7.65 ms, total: 131 ms
Wall time: 4.79 s


9.905162372589585

## 2010 data create

In [40]:
# read data in without dtypes
ddf = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2010-*.csv",
    parse_dates=["pickup_datetime", "dropoff_datetime"],
    on_bad_lines="skip",
    dtype={
        'tolls_amount': 'float64',
        "store_and_fwd_flag": "object",
    },
)

In [41]:
ddf.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,VTS,2010-01-26 07:41:00,2010-01-26 07:45:00,1,0.75,-73.956778,40.767750,1,NaN,-73.965957,40.765232,CAS,4.5,0.0,0.5,0.00,0.0,5.00
1,DDS,2010-01-30 23:31:00,2010-01-30 23:46:12,1,5.90,-73.996118,40.763932,1,NaN,-73.981512,40.741193,CAS,15.3,0.5,0.5,0.00,0.0,16.30
2,DDS,2010-01-18 20:22:20,2010-01-18 20:38:12,1,4.00,-73.979673,40.783790,1,NaN,-73.917852,40.878560,CAS,11.7,0.5,0.5,0.00,0.0,12.70
3,VTS,2010-01-09 01:18:00,2010-01-09 01:35:00,2,4.70,-73.977922,40.763997,1,NaN,-73.923908,40.759725,CAS,13.3,0.5,0.5,0.00,0.0,14.30
4,CMT,2010-01-18 19:10:14,2010-01-18 19:17:07,1,0.60,-73.990924,40.734682,1,0,-73.995511,40.739088,Cre,5.3,0.0,0.5,0.87,0.0,6.67


In [29]:
%%time
# calculate .describe() to check for dtype issues
ddf.describe().compute()

CPU times: user 1.76 s, sys: 173 ms, total: 1.94 s
Wall time: 1min 55s


,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,dropoff_longitude,dropoff_latitude,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
count,1.689944e+08,1.689944e+08,1.689944e+08,1.689944e+08,1.689944e+08,1.689942e+08,1.689942e+08,1.689944e+08,1.689944e+08,1.689944e+08,1.689944e+08,1.689944e+08,1.689944e+08
mean,1.674221e+00,5.864681e+00,-7.239051e+01,3.988347e+01,1.032463e+00,-7.241661e+01,3.989821e+01,9.844589e+00,3.221727e-01,4.955940e-01,7.605548e-01,-7.496016e-02,1.135107e+01
std,1.300666e+00,5.409394e+03,1.102328e+01,7.053625e+00,4.236116e-01,1.089876e+01,7.027043e+00,1.664848e+03,3.693486e-01,1.345498e-01,1.735413e+02,2.336194e+03,2.873962e+03
min,0.000000e+00,-2.147483e+07,-3.509015e+03,-3.579139e+03,0.000000e+00,-3.579139e+03,-3.538432e+03,-2.147481e+07,-1.000000e+00,-1.000000e+00,-1.677720e+06,-2.147484e+07,-2.147483e+07
25%,1.000000e+00,1.090000e+00,-7.399139e+01,4.073673e+01,1.000000e+00,-7.399071e+01,4.073553e+01,5.700000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,7.000000e+00
50%,1.000000e+00,1.800000e+00,-7.398101e+01,4.075392e+01,1.000000e+00,-7.397925e+01,4.075437e+01,8.100000e+00,5.000000e-01,5.000000e-01,0.000000e+00,0.000000e+00,9.540000e+00
75%,3.000000e+00,3.250000e+00,-7.396582e+01,4.076844e+01,1.000000e+00,-7.396257e+01,4.076894e+01,1.210000e+01,5.000000e-01,5.000000e-01,1.600000e+00,0.000000e+00,1.380000e+01
max,2.550000e+02,1.620163e+07,3.569931e+03,3.377993e+03,2.210000e+02,3.443651e+03,3.510381e+03,9.396007e+04,6.157800e+02,1.311220e+03,9.380200e+02,5.510070e+03,9.396057e+04


In [45]:
# verify dtypes and column names are identical for 2009 and 2010
dtypes_2010 = ddf.dtypes
dtypes_2009 == dtypes_2010

vendor_id             True
pickup_datetime       True
dropoff_datetime      True
passenger_count       True
trip_distance         True
pickup_longitude      True
pickup_latitude       True
rate_code             True
store_and_fwd_flag    True
dropoff_longitude     True
dropoff_latitude      True
payment_type          True
fare_amount           True
surcharge             True
mta_tax               True
tip_amount            True
tolls_amount          True
total_amount          True
dtype: bool

In [47]:
# repartition and write 2010 data to Parquet 
ddf.repartition(partition_size="100MB").to_parquet(
    "s3://coiled-datasets/dask-book/nyc-tlc/2010",
    engine="pyarrow",
    compression="snappy",
    write_metadata_file=False,
)

KeyboardInterrupt: 

In [ ]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/dask-book/nyc-tlc/2010", 
    engine="pyarrow"
)

In [ ]:
# check datatypes are stored correctly in Parquet file
dtypes_2010 == ddf.dtypes

## 2011 data create

In [37]:
ddf = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2011-*.csv",
    parse_dates=["pickup_datetime", "dropoff_datetime"],
    dtype={
        "tip_amount": "float64",
        "tolls_amount": "float64",
        # "vendor_name": "string[pyarrow]",
        # "Payment_Type": "string[pyarrow]",
    },
)

In [38]:
ddf.dtypes

vendor_id                     object
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count                int64
trip_distance                float64
pickup_longitude             float64
pickup_latitude              float64
rate_code                      int64
store_and_fwd_flag            object
dropoff_longitude            float64
dropoff_latitude             float64
payment_type                  object
fare_amount                  float64
surcharge                    float64
mta_tax                      float64
tip_amount                   float64
tolls_amount                 float64
total_amount                 float64
dtype: object

In [39]:
# verify data types are identical to those in 2010 dataset
dtypes_2010 == ddf.dtypes

vendor_id             True
pickup_datetime       True
dropoff_datetime      True
passenger_count       True
trip_distance         True
pickup_longitude      True
pickup_latitude       True
rate_code             True
store_and_fwd_flag    True
dropoff_longitude     True
dropoff_latitude      True
payment_type          True
fare_amount           True
surcharge             True
mta_tax               True
tip_amount            True
tolls_amount          True
total_amount          True
dtype: bool

In [17]:
ddf.repartition(partition_size="100MB").to_parquet(
    "s3://coiled-datasets/dask-book/nyc-tlc/2011",
    engine="pyarrow",
    compression="snappy",
    write_metadata_file=False,
)

[None]

In [12]:
ddf = dd.read_parquet("s3://coiled-datasets/nyc-tlc/2011", engine="pyarrow")

In [13]:
actual_dtypes_2011 = ddf.dtypes

In [15]:
type(actual_dtypes_2011)

pandas.core.series.Series

In [21]:
actual_dtypes_2011

vendor_id                     object
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count                int64
trip_distance                float64
pickup_longitude             float64
pickup_latitude              float64
rate_code                      int64
store_and_fwd_flag            object
dropoff_longitude            float64
dropoff_latitude             float64
payment_type                  object
fare_amount                  float64
surcharge                    float64
mta_tax                      float64
tip_amount                   float64
tolls_amount                 float64
total_amount                 float64
dtype: object

In [20]:
actual_dtypes_2011 == actual_dtypes_2010

vendor_id             True
pickup_datetime       True
dropoff_datetime      True
passenger_count       True
trip_distance         True
pickup_longitude      True
pickup_latitude       True
rate_code             True
store_and_fwd_flag    True
dropoff_longitude     True
dropoff_latitude      True
payment_type          True
fare_amount           True
surcharge             True
mta_tax               True
tip_amount            True
tolls_amount          True
total_amount          True
dtype: bool

## 2012 Data Create

In [22]:
ddf = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2012-*.csv",
    parse_dates=["pickup_datetime", "dropoff_datetime"],
    dtype={
        "tip_amount": "float64",
        "tolls_amount": "float64",
    },
)

In [23]:
ddf.dtypes

vendor_id                     object
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count                int64
trip_distance                float64
pickup_longitude             float64
pickup_latitude              float64
rate_code                      int64
store_and_fwd_flag            object
dropoff_longitude            float64
dropoff_latitude             float64
payment_type                  object
fare_amount                  float64
surcharge                    float64
mta_tax                      float64
tip_amount                   float64
tolls_amount                 float64
total_amount                 float64
dtype: object

In [24]:
ddf.dtypes == actual_dtypes_2010

vendor_id             True
pickup_datetime       True
dropoff_datetime      True
passenger_count       True
trip_distance         True
pickup_longitude      True
pickup_latitude       True
rate_code             True
store_and_fwd_flag    True
dropoff_longitude     True
dropoff_latitude      True
payment_type          True
fare_amount           True
surcharge             True
mta_tax               True
tip_amount            True
tolls_amount          True
total_amount          True
dtype: bool

In [25]:
ddf.repartition(partition_size="100MB").to_parquet(
    "s3://coiled-datasets/nyc-tlc/2012",
    engine="pyarrow",
    compression="snappy",
    write_metadata_file=False,
)

[None]

## 2013 Data Create

In [26]:
ddf = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2013-*.csv",
    parse_dates=["pickup_datetime", "dropoff_datetime"],
    dtype={
        "tip_amount": "float64",
        "tolls_amount": "float64",
    },
)

In [27]:
ddf.dtypes == actual_dtypes_2010

vendor_id             True
pickup_datetime       True
dropoff_datetime      True
passenger_count       True
trip_distance         True
pickup_longitude      True
pickup_latitude       True
rate_code             True
store_and_fwd_flag    True
dropoff_longitude     True
dropoff_latitude      True
payment_type          True
fare_amount           True
surcharge             True
mta_tax               True
tip_amount            True
tolls_amount          True
total_amount          True
dtype: bool

In [28]:
len(ddf.dtypes)

18

In [29]:
ddf.dtypes

vendor_id                     object
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count                int64
trip_distance                float64
pickup_longitude             float64
pickup_latitude              float64
rate_code                      int64
store_and_fwd_flag            object
dropoff_longitude            float64
dropoff_latitude             float64
payment_type                  object
fare_amount                  float64
surcharge                    float64
mta_tax                      float64
tip_amount                   float64
tolls_amount                 float64
total_amount                 float64
dtype: object

In [30]:
ddf.repartition(partition_size="100MB").to_parquet(
    "s3://coiled-datasets/nyc-tlc/2013",
    engine="pyarrow",
    compression="snappy",
    write_metadata_file=False,
)

[None]

## 2019 Data Create

In [4]:
ddf = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype={
        "RatecodeID": "float64",
        "VendorID": "float64",
        "passenger_count": "float64",
        "payment_type": "object",
    },
)

In [6]:
ddf.dtypes

VendorID                        float64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                     object
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
dtype: object

In [16]:
ddf.repartition(partition_size="100MB").to_parquet(
    "s3://coiled-datasets/nyc-tlc/2019",
    engine="pyarrow",
    compression="snappy",
    write_metadata_file=False,
)

[None]

In [7]:
ddf.repartition(partition_size="100MB").to_parquet(
    "s3://coiled-datasets/nyc-tlc-with-metadata/2019",
    engine="pyarrow",
    compression="snappy",
    write_metadata_file=True,
)

[None]

## 2020 Data

In [17]:
ddf = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2020-*.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype={
        "RatecodeID": "float64",
        "VendorID": "float64",
        "passenger_count": "float64",
        "payment_type": "object",
    },
)

In [18]:
ddf.dtypes

VendorID                        float64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                     object
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                      int64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
dtype: object

## Data queries

In [86]:
ddf2009 = dd.read_parquet(
    "s3://coiled-datasets/nyc-tlc/2009",
    engine="pyarrow",
)

In [87]:
len(ddf2009)

OSError: [Errno 22] Bad Request

In [80]:
ddf2010 = dd.read_parquet(
    "s3://coiled-datasets/nyc-tlc/2010",
    engine="pyarrow",
)

In [81]:
ddf2011 = dd.read_parquet(
    "s3://coiled-datasets/nyc-tlc/2011",
    engine="pyarrow",
)

In [82]:
ddf2012 = dd.read_parquet(
    "s3://coiled-datasets/nyc-tlc/2012",
    engine="pyarrow",
)

In [83]:
ddf2013 = dd.read_parquet(
    "s3://coiled-datasets/nyc-tlc/2013",
    engine="pyarrow",
)

In [84]:
ddf = dd.concat([ddf2009, ddf2010, ddf2011, ddf2012, ddf2013])

In [85]:
len(ddf)

PermissionError: The provided token has expired.